In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

# Кластеризация

Наша задача - предсказать есть диабет у индивида или нет. В качестве таргета - колонка Diabetes. В нем три различных значения: `0`, `1`, `2`. `0` означает, что наблюдаемой здоров, `1` значит, что есть риск диабета, `2` означает наличие диабета. В качестве признаков будем использовать пол, количество лет в США, доход семьи и некоторые показатели, измеренные медицинскими работниками.  

**Задание 1.** В этой части ДЗ попробуем использовать кластеризацию как инструмент при проведении моделирования в задаче классификации

In [ ]:
df = pd.read_csv('datahw21.csv', index_col='Unnamed: 0')

df.head()

In [ ]:
### Посмотрим как устроены данные
### Изобразим корреляционную матрицу

colormap = plt.cm.viridis
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), linewidths=0.1, vmax=1.0, square=True, cmap=colormap, annot=True)

In [ ]:
### Разделим выборку на трейн-тест

data = df.drop(['Diabetes'], axis=1)
target = df[['Diabetes']]

X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target, 
                                                    test_size=0.25,
                                                    random_state=1)

Для того, чтобы использовать K-means, лучше будет отнормировать данные. 

In [ ]:
### Воспользуемся StandardScaler

### Your code is here




Обучим K-Means с параметрами `n_clusters` = 3, `tol` = 0.0005. Выбор параметров обусловлен тем, что у нас три возможных значения таргета. Но в целом основной подход подбора количества кластеров - по кривой зависимости внутрикластерного и межкластерного расстояний от количества кластеров.

Установите `random_state` = 1

In [ ]:
### Your code is here



Посчитаем качество на изначальных данных(нормированных). Для этого обучите с дефолтными параметрами `RandomForestClassifier`, `LogisticRegression`, `LinearSVC`. Там, где нужно, установите `random_state` = 1. (1б)

In [ ]:
### Your code is here




Добавьте в признаковое описание номер кластера и посчитайте качество с новым признаком! Стало ли качество хоть сколько-то лучше? (1б)

In [ ]:
### Your code is here




Посчитаем расстояния от объектов до центров кластеров. Для этого воспользуемся методом `transform` обученного класса kmeans.

Обучим и посчитаем метрики исключительно на расстояниях до центра. Убедимся, что такой подход имеет право на существование, если данные позволяют, то качество не сильно должно пострадать. А в каких-то случаях может оказаться даже лучше! Таким образом можно снижать размерность данных. (2б)

In [ ]:
### Your code is here




**Задание 2** Задача кластеризации может использоваться не только для специфических задач группировки данных, но и для оптимизации других методов. Вы уже знаете, что одна из основных проблем kNN в скорости его предсказания. В этом задании попробуем ускорить работу kNN с помощью кластеризации, не теряя при этом сильно в качестве.

Сначала загрузим уже известные вам данные клиентов страховой компании.

In [ ]:
data = pd.read_csv('processed_vehicle_inssurance.csv')
data.head()

In [ ]:
### Разделим выборку на трейн-тест

from sklearn.model_selection import train_test_split
X = data.drop('Response', axis=1)[:25000]
y = data['Response'][:25000]

X_train, X_test, y_train, y_test = train_test_split(X, y.values,
                                                    random_state=0,
                                                    test_size=0.2)

In [ ]:
### Нормируем данные

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Обучите обычный kNN с одним соседом и измерьте качество, например, взвешенную f-меру, чтобы потом сравнить с нашей реализацией.

In [ ]:
### Your code is here



In [ ]:
### Your code is here




Идея ускорения kNN заключается в том, чтобы разбить признаковое пространство (то есть столбцы, а не объекты-строки!) на несколько блоков и кластеризовать каждый блок по-отдельности.

Далее нужно будет заменить в обучающей выборке объекты на их некоторое представление, а именно вместо каждого блока вставить центр кластера, к которому принадлежит эта часть исходного объекта.

Затем подсчет расстояний на этапе применения производится по блокам. Главный плюс алгоритма в том, что у нас уже будут иметься предподсчитанные расстояния по блокам до всех центров кластеров.

Этот алгоритм называется **Product Quantization**. 


Сам алгоритм:

1. Разделите обучающую и контрольную выборки на блоки: равномерно по индексам столбцов.


2. На каждом блоке обучите K-Means и примените transform к соотв. блоку контрольной выборки.


3. Посчитайте расстояния от каждого обучающего объекта до каждого объекта из контрольной выборки
(это вы должны сделать, используя матрицы из предыдущего пункта)


4. Определите для каждого тестового объекта k ближайших

В нашей реализации будем использовать следующие значения параметров: \
`m_blocks` = 5 \
`n_clusters` = 100 \
`k` = 1

In [ ]:
def product_quantization(X_train, X_test, m_blocks=5, n_clusters=100):
    dist_table = np.zeros([X_test.shape[0], n_clusters, m_blocks])
    X_train_clusters = np.zeros([X_train.shape[0], m_blocks])
    
    for i in range(m_blocks):
        ### Вырежьте блок из обучающей и контрольной выборок
        
        
        ### Обучите K-Means и примените transform на тестовой выборке
        ### Положите посчитанные расстояния до центров в матрицу dist_table
        
        
        ### Положите метки кластеров в матрицу X_train_clusters
        
        
    return X_train_clusters, dist_table

**Совет на будущее в практике:** Обучайте все на подвыборке данных, так как итоговая матрица при подсчете kNN будет занимать очень много оперативной памяти. 

In [ ]:
%%time
X_train_clusters, dist_table = product_quantization(X_train, X_test, 
                                                    m_blocks=5, 
                                                    n_clusters=100)

Теперь нам нужно для каждого блока из m_blocks просуммировать расстояния из полученной таблицы. Сделать это можно в цикле, отфильтровав нужные значения из X_train_clusters и таблицы квадратов расстояний dist_table по номеру блока m_blocks.

То есть для каждого объекта в зависимости от его кластера (n_clusters) нам нужно отобрать столбец в таблице квадратов расстояний dist_table, соответствующий этому кластеру (в столбце будут лежать расстояния всех объектов до центра этого кластера), не забыв про то, что матрица dist_table трехмерная размера 5000х100х5 и нужно отобрать еще и нужный блок из m_blocks

In [ ]:
%%time
distances = np.zeros([sumsq_table.shape[0], X_train.shape[0]])
m_blocks = 5

for b in range(m_blocks):
        distances += sumsq_table[:, X_train_clusters[:, b],b] 

y_pred = y_train[np.argmin(distances, axis=1)]

Наша реализация на python работает медленнее по сравнению с библиотечным kNN. Однако реализация на более низкоуровневых языках программирования (C++) и на большом количестве данных данный метод на самом деле позволяет ускорять подсчет расстояний.